## Demo notebook for New Zealand Wildlife Thermal Imaging

This notebook demonstrates basic interaction with video clips from the [New Zealand Wildlife Thermal Imaging ](https://lila.science/datasets/new-zealand-wildlife-thermal-imaging/) dataset, which contains ~120k thermal video clips.  Data was provided by the [Cacophony Project](https://cacophony.org.nz/).

This notebook assumes you've downloaded the metadata and all the videos; tell the notebook where you put them in the next cell.  You only need the HDF files if you want to run the last cell.

### Constants and imports

In [ ]:
import os
import sys
import json
import random

from collections import defaultdict
from tqdm import tqdm
from IPython.display import Video

import matplotlib.pyplot as plt

random.seed(0)

# Change this to point to the folder where you placed all the data
base_folder = os.path.expanduser('~/tmp/new-zealand-wildlife-thermal-imaging')

# The HDF files are only required if you want to run the last cell, but if you downloaded
# them, change this to point to the folder where you placed the HDF files.
hdf_folder = '/bigdata/home/sftp/cacophony-ferraro_/data/cacophony-thermal/'

### Make sure all the files are where we expect them to be

In [ ]:
main_metadata_filename = os.path.join(base_folder,'new-zealand-wildlife-thermal-imaging.json')
individual_metadata_folder = os.path.join(base_folder,'individual-metadata')
video_folder = os.path.join(base_folder,'videos')

assert os.path.isdir(base_folder)
assert os.path.isdir(video_folder)
assert os.path.isdir(individual_metadata_folder)
assert os.path.isfile(main_metadata_filename)

### Open the metadata file, print some statistics

In [ ]:
with open(main_metadata_filename,'r') as f:
    metadata = json.load(f)

# Print basic info about the dataset    
for k in metadata['info']:
    print('{}: {}'.format(k,metadata['info'][k]))
    
# From now on, we only care about the 'clips' field    
all_clip_metadata = metadata['clips']
print('Read metadata for {} clips\n'.format(len(all_clip_metadata)))

# Count the number of videos associated with each label    
label_to_video_count = defaultdict(int)

for clip_metadata in tqdm(all_clip_metadata,file=sys.stdout):
    
    if clip_metadata['error'] is not None:
        continue
     
    # This will be a list of labels attached to this video
    labels_this_clip = set()
    
    # Labels are actually assigned to *track* (individual moving blobs), so
    # look at all the tracks that were labeled on this video.
    for track_info in clip_metadata['tracks']:
        for tag in track_info['tags']:
            tag_label = tag['label']
            labels_this_clip.add(tag_label)
    
    for label in labels_this_clip:
        label_to_video_count[label] += 1
              
# Print labels in descending order of frequency            
label_to_video_count = {k: v for k, v in sorted(label_to_video_count.items(), 
                                                key=lambda item: item[1], reverse=True)}

print('\nLabels:\n')

for label in label_to_video_count:
    print('{}: {}'.format(label,label_to_video_count[label]))

## Choose a clip to tinker with

Run this cell and subsequent cells multiple times to tinker with multiple videos.  By default we fix the random seed in the first cell of this notebook, so you will get the same sequence every time you run this notebook.

In [ ]:
# By default, pick a clip at random
i_clip = random.randint(0,len(all_clip_metadata)-1)

# If you want to pick a specific clip by ID...
if False:
    def find_clip(clip_id):
        return [c['id'] for c in all_clip_metadata].index(clip_id)    
    i_clip = find_clip(409532)
    
clip = all_clip_metadata[i_clip]
assert clip['error'] is None, 'Oops, you had the bad luck of randomly choosing an invalid clip'

print('Tinkering with clip {}:\n'.format(i_clip))

# Print information about this clip
for k in clip:
    if k == 'tracks':
        print('Tracks:')
        for track in clip['tracks']:
            print('  ' + str(track))
    elif k == 'calibration_frames':
        print('N calibration frame: {}'.format(len(clip['calibration_frames'])))
    else:
        print('{}: {}'.format(k,clip[k]))

### Play the videos file associated with this clip

#### First the normalized (but not background-corrected) video

In [ ]:
video_filename = os.path.join(video_folder,clip['video_filename'])
assert os.path.isfile(video_filename)

print('Playing non-background-corrected video ({}):'.format(video_filename))
Video(video_filename,embed=True,width=500)

### Now the background-corrected video

In [ ]:
filtered_video_filename = os.path.join(video_folder,clip['filtered_video_filename'])
assert os.path.isfile(filtered_video_filename)

print('Playing background-corrected video ({}):'.format(filtered_video_filename))
Video(filtered_video_filename,embed=True,width=500)

### Render the tracks associated with this clip

These should match whatever moving blobs you just saw in the video.  These are not stored in the main metadata file (because they make it very large, and you don't *need* this information for most things you might do with this dataset), so we need to load the metadata file for this specific file.

In [ ]:
clip_metadata_filename = clip['metadata_filename']
with open(os.path.join(individual_metadata_folder,clip_metadata_filename)) as f:
    clip_metadata_with_position = json.load(f)

# Print some basic metadata about each track, and plot them
for track in clip_metadata_with_position['tracks']:
    for k in track:
        if k == 'points':
            print('{} points'.format(len(track['points'])))
        else:
            print('{}: {}'.format(k,track[k]))

# A list of (x[],y[]) tracks
track_coords = []
for track in clip_metadata_with_position['tracks']:
    # These are organized as x/y/frame    
    points = track['points']
    x = [p[0] for p in points]
    y = [(clip['height'] - p[1]) for p in points]
    track_coords.append((x,y))
    
import matplotlib.pyplot as pl
for t in track_coords:
    x = t[0]
    y = t[1]
    plt.plot(x,y,'.')
plt.xlim([0, clip['width']])
plt.ylim([0, clip['height']])

### Explore the HDF file associated with this clip

It's unlikely you'll need anything from the HDF file if you're doing Machine Learning Stuff (tm) with this dataset, but just in case...

In [ ]:
import h5py
import numpy as np

h5f = h5py.File(os.path.join(hdf_folder,clip['hdf_filename']),'r')

# Print the metadata in the HDF file
clip_attrs = h5f.attrs
for k in clip_attrs:
    if isinstance(clip_attrs[k],np.ndarray):
        print('{}: array of length {}'.format(k,len(clip_attrs[k])))
    else:
        print('{}: {}'.format(k,clip_attrs[k]))
        
# This is where the actual frames are, as an array of size nframes,x,y
thermal_frames = h5f['frames']['thermals']

assert clip_attrs.get('num_frames') == thermal_frames.shape[0]

# Show an image of the first frame
thermal_frames_array = np.array(thermal_frames)
sample_frame = thermal_frames_array[0]
plt.imshow(sample_frame,cmap='jet')